## Prepare imports

In [1]:
from datasets.tu import TUBZRConfig, TUDataModule
import torch
import matplotlib.pyplot as plt
from omegaconf import OmegaConf


from models.ectencoder_modelnet import BaseModel as EctEncoder
from layers.ect import EctLayer, EctConfig

from metrics.metrics import get_mse_metrics
from metrics.accuracies import compute_mse_accuracies
from metrics.loss import compute_mse_loss_fn
from directions import generate_3d_directions


DEVICE = "cuda:0"

config = OmegaConf.load("./configs/config_encoder_bzr.yaml")


dm = TUDataModule(TUBZRConfig(use_node_attr=True))

print(len(dm.test_ds))

# for batch in dm.test_dataloader():
#     print(batch.y)

56


## Load Models and Data

In [2]:


layer = EctLayer(
    EctConfig(
        num_thetas=config.layer.ect_size,
        bump_steps=config.layer.ect_size,
        normalized=True,
        device=DEVICE,
    ),
    v=generate_3d_directions(config.layer.ect_size, DEVICE),
)

# Load the encoder 

ect_encoder_litmodel = EctEncoder.load_from_checkpoint(
    f"./trained_models/ectencoder_bzr.ckpt",
    layer=layer,
    ect_size=config.layer.ect_size,
    hidden_size=config.encodermodel.hidden_size,
    num_pts=config.encodermodel.num_pts,
    num_dims=config.encodermodel.num_dims,
    learning_rate=config.encodermodel.learning_rate,
).to(DEVICE)


## Generate an ECT and use VAE as autoencoder to recreate the ECT

In [3]:
for features in dm.test_dataloader():
    break

In [4]:

features.to(DEVICE)
ect = layer(features,features.batch).unsqueeze(1)

with torch.no_grad():
    decoded = ect_encoder_litmodel.model.forward(ect)

In [5]:
import pyvista as pv


pl = pv.Plotter(shape=(2, 8), window_size=[1600,400],border=False,polygon_smoothing=True,off_screen=True)

batch = decoded.cpu().detach().numpy()


for col in range(8):
    points = batch[col].reshape(-1, 3)
    pl.subplot(0, col)
    actor = pl.add_points(
        features[col].x.cpu().detach().numpy(),
        style="points",
        emissive=False,
        show_scalar_bar=False,
        render_points_as_spheres=True,
        scalars=points[:, 2],
        point_size=5,
        ambient=0.2, 
        diffuse=0.8, 
        specular=0.8,
        specular_power=40, 
        smooth_shading=True
    )
    pl.subplot(1, col)
    actor = pl.add_points(
        points,
        style="points",
        emissive=False,
        show_scalar_bar=False,
        render_points_as_spheres=True,
        scalars=points[:, 2],
        point_size=5,
        ambient=0.2, 
        diffuse=0.8, 
        specular=0.8,
        specular_power=40, 
        smooth_shading=True
    )


pl.background_color = "w"
pl.link_views()
pl.camera_position = "yz"
pos = pl.camera.position
pl.camera.position = (pos[0],pos[1],pos[2]+3)
pl.camera.azimuth = -45
pl.camera.elevation = 10

# create a top down light
light = pv.Light(position=(0, 0, 3), positional=True,
                cone_angle=50, exponent=20, intensity=.2)
pl.add_light(light)
pl.camera.zoom(1.3)
# pl.screenshot("./figures/reconstructed_topological/reconstructed_pointcloud.png",transparent_background=True,scale=2)
pl.show()
# path = pl.generate_orbital_path(n_points=64, shift=2, factor=3.0)
# pl.open_gif("./figures/reconstructed_topological/orbit_cloud.gif")
# pl.orbit_on_path(path, write_frames=True)
# pl.close()


Widget(value='<iframe src="http://localhost:59445/index.html?ui=P_0x12d382b40a0_0&reconnect=auto" class="pyvis…

## Sample from VAE and reconstruct points 

In [6]:
batch = features.x.cpu().detach().numpy()
batch

array([[-0.48657328,  0.4617581 ,  0.01141672],
       [-0.528499  ,  0.21029627,  0.04302314],
       [-0.33298758,  0.04703971,  0.07985406],
       ...,
       [ 0.24542014,  0.29359546, -0.03490202],
       [-0.0442293 ,  0.0681807 , -0.03583009],
       [-0.26892897, -0.51788855,  0.10893377]], dtype=float32)

In [7]:
import pyvista as pv

pl = pv.Plotter(shape=(8, 8), window_size=[1600, 1600],border=False,polygon_smoothing=True,off_screen=True)




for row in range(8):
    for col in range(8):
        points = features[8*row + col].x.reshape(-1, 3).numpy()
        pl.subplot(row, col)
        actor = pl.add_points(
            points,
            style="points",
            emissive=False,
            show_scalar_bar=False,
            render_points_as_spheres=True,
            scalars=points[:, 2],
            point_size=5,
            ambient=0.2, 
            diffuse=0.8, 
            specular=0.8,
            specular_power=40, 
            smooth_shading=True
        )


pl.background_color = "w"
pl.link_views()
pl.camera_position = "yz"
pos = pl.camera.position
pl.camera.position = (pos[0],pos[1],pos[2]+3)
pl.camera.azimuth = -45
pl.camera.elevation = 10

# create a top down light
light = pv.Light(position=(0, 0, 3), positional=True,
                cone_angle=50, exponent=20, intensity=.2)
pl.add_light(light)
pl.camera.zoom(1.3)
# pl.screenshot("./figures/reconstructed_topological/reconstructed_pointcloud.png",transparent_background=True,scale=2)
pl.show()
# path = pl.generate_orbital_path(n_points=64, shift=2, factor=3.0)
# pl.open_gif("./figures/reconstructed_topological/orbit_cloud.gif")
# pl.orbit_on_path(path, write_frames=True)
pl.close()

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.